In [1]:
from bs4 import BeautifulSoup as bs #pip3 install beautifulsoup4

xml_file="data/br-a01.xml"
with open(xml_file, "r") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.read()
    #Combine the lines in the list into a string
    #content = "".join(content)
    soup = bs(content,"lxml")

In [2]:
soup.__dict__.keys()

dict_keys(['element_classes', 'builder', 'is_xml', 'known_xml', '_namespaces', 'parse_only', 'markup', 'original_encoding', 'declared_html_encoding', 'contains_replacement_characters', 'parser_class', 'name', 'namespace', 'prefix', 'attrs', 'contents', 'parent', 'previous_element', 'next_element', 'next_sibling', 'previous_sibling', 'hidden', 'can_be_empty_element', 'cdata_list_attributes', 'preserve_whitespace_tags', 'current_data', 'currentTag', 'tagStack', 'open_tag_counter', 'preserve_whitespace_tag_stack', 'string_container_stack', '_most_recent_element'])

In [3]:
from pprint import pprint
pprint(soup.__dict__)

{'_most_recent_element': '\n',
 '_namespaces': {'xml': 'http://www.w3.org/XML/1998/namespace'},
 'attrs': {},
 'builder': <bs4.builder._lxml.LXMLTreeBuilder object at 0x7f06d4729d10>,
 'can_be_empty_element': False,
 'cdata_list_attributes': {'*': ['class', 'accesskey', 'dropzone'],
                           'a': ['rel', 'rev'],
                           'area': ['rel'],
                           'form': ['accept-charset'],
                           'icon': ['sizes'],
                           'iframe': ['sandbox'],
                           'link': ['rel', 'rev'],
                           'object': ['archive'],
                           'output': ['for'],
                           'td': ['headers'],
                           'th': ['headers']},
 'contains_replacement_characters': False,
 'contents': ['xml version="1.0" encoding="UTF-8"?',
              'SimpleWsdDoc SYSTEM "simple-wsd-doc.dtd"',
              <html><body><simplewsddoc name="br-a01">
<word break_level="NO_BR

 'tagStack': [<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE SimpleWsdDoc SYSTEM "simple-wsd-doc.dtd">
<html><body><simplewsddoc name="br-a01">
<word break_level="NO_BREAK" text="The"></word>
<word break_level="SPACE_BREAK" text="Fulton"></word>
<word break_level="SPACE_BREAK" text="County"></word>
<word break_level="SPACE_BREAK" text="Grand"></word>
<word break_level="SPACE_BREAK" text="Jury"></word>
<word break_level="SPACE_BREAK" lemma="say" pos="VERB" sense="/dictionary/sense/en_us_NOAD3e_2012/m_en_us1287486.001" text="said"></word>
<word break_level="SPACE_BREAK" text="Friday"></word>
<word break_level="SPACE_BREAK" text="an"></word>
<word break_level="SPACE_BREAK" text="investigation"></word>
<word break_level="SPACE_BREAK" text="of"></word>
<word break_level="SPACE_BREAK" text="Atlanta"></word>
<word break_level="NO_BREAK" text="'s"></word>
<word break_level="SPACE_BREAK" lemma="recent" pos="ADJ" sense="/dictionary/sense/en_us_NOAD3e_2012/m_en_us1283413.001" text="recent"></wor

In [4]:
pprint(soup.contents[2])

<html><body><simplewsddoc name="br-a01">
<word break_level="NO_BREAK" text="The"></word>
<word break_level="SPACE_BREAK" text="Fulton"></word>
<word break_level="SPACE_BREAK" text="County"></word>
<word break_level="SPACE_BREAK" text="Grand"></word>
<word break_level="SPACE_BREAK" text="Jury"></word>
<word break_level="SPACE_BREAK" lemma="say" pos="VERB" sense="/dictionary/sense/en_us_NOAD3e_2012/m_en_us1287486.001" text="said"></word>
<word break_level="SPACE_BREAK" text="Friday"></word>
<word break_level="SPACE_BREAK" text="an"></word>
<word break_level="SPACE_BREAK" text="investigation"></word>
<word break_level="SPACE_BREAK" text="of"></word>
<word break_level="SPACE_BREAK" text="Atlanta"></word>
<word break_level="NO_BREAK" text="'s"></word>
<word break_level="SPACE_BREAK" lemma="recent" pos="ADJ" sense="/dictionary/sense/en_us_NOAD3e_2012/m_en_us1283413.001" text="recent"></word>
<word break_level="SPACE_BREAK" text="primary"></word>
<word break_level="SPACE_BREAK" text="election

In [5]:
result = soup.find_all("word")
len(result)

2255

In [6]:
text=result[0].get("text")
sense=result[0].get("sense")
print(f" text={text} sense={sense}")

 text=The sense=None


## Write a function to get all words with there sense

In [7]:
def get_words_with_sense(result):
    """
    """
    data=[]
    for n in range(len(result)):      
        text=result[n].get("text")
        sense=result[n].get("sense")
        if not (sense==None):
            data.append({"word":text,"sense":sense})
    return data
    
data=get_words_with_sense(result)
        

In [8]:
import pandas as pd

df=pd.DataFrame.from_dict(data)
df.head()

len(df['sense'].unique().tolist())

246

### Write a function that given a word with sense label give it context as the two previous and following tokens (words or punctuations)

In [57]:
import pandas as pd

def get_target_word_context(result,context_size):
    """
    """
    # Put xml file content into a dataframe
    data=[]
    for n in range(len(result)):      
        text=result[n].get("text")
        sense=result[n].get("sense")
        data.append({"word":text,"sense":sense})
    df=pd.DataFrame.from_dict(data)
    
    # Keep word with sense
    sense_df = df[df['sense'].notnull()]
    
    # Extract the context (context_size words before and after) of word with sense 
    context = []
    for n in range(sense_df.shape[0]):
        m=max(sense_df.index[n]-context_size,0)
        M=min(sense_df.index[n]+context_size+1,sense_df.shape[0])
        indices=[l for l in range(m,M) if not(l==sense_df.index[n])]
        context.append(df['word'].iloc[indices].tolist())
    sense_df['context']=context
    
    return df,sense_df
    
df, sense_df=get_target_word_context(result,context_size=5)

print(df.head(20))
print(sense_df.head(2))


             word                                              sense
0             The                                               None
1          Fulton                                               None
2          County                                               None
3           Grand                                               None
4            Jury                                               None
5            said  /dictionary/sense/en_us_NOAD3e_2012/m_en_us128...
6          Friday                                               None
7              an                                               None
8   investigation                                               None
9              of                                               None
10        Atlanta                                               None
11             's                                               None
12         recent  /dictionary/sense/en_us_NOAD3e_2012/m_en_us128...
13        primary                 

/home/sba/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [59]:
sense_df.head(20)

,word,sense,context
5,said,/dictionary/sense/en_us_NOAD3e_2012/m_en_us128...,"[The, Fulton, County, Grand, Jury, Friday, an,..."
12,recent,/dictionary/sense/en_us_NOAD3e_2012/m_en_us128...,"[an, investigation, of, Atlanta, 's, primary, ..."
28,further,/dictionary/sense/en_us_NOAD3e_2012/m_en_us124...,"[took, place, ., The, jury, said, in, term, en..."
29,said,/dictionary/sense/en_us_NOAD3e_2012/m_en_us128...,"[place, ., The, jury, further, in, term, end, ..."
41,had,/dictionary/sense/en_us_NOAD3e_2012/m_en_us125...,"[City, Executive, Committee, ,, which, over-al..."
43,charge,/dictionary/sense/en_us_NOAD3e_2012/m_en_us123...,"[Committee, ,, which, had, over-all, of, the, ..."
62,manner,/dictionary/sense/en_us_NOAD3e_2012/m_en_us126...,"[of, Atlanta, "", for, the, in, which, the, ele..."
68,conducted,/dictionary/sense/en_us_NOAD3e_2012/m_en_us123...,"[in, which, the, election, was, ., The, Septem..."
77,charged,/dictionary/sense/en_us_NOAD3e_2012/m_en_us123...,"[October, term, jury, had, been, by, Fulton, S..."
96,primary,/dictionary/sense/en_us_NOAD3e_2012/m_en_us128...,"[irregularities, "", in, the, hard-fought, whic..."
